In [1]:
import os
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

ModuleNotFoundError: No module named 'sklearn'

In [3]:
image_sizes = [128, 128, 128]

data_dir = '../data/raw/'

In [10]:
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
mask_files = os.listdir(f'{data_dir}/segmentations')
df_mask = pd.DataFrame({
    'mask_file': mask_files,
})
df_mask['StudyInstanceUID'] = df_mask['mask_file'].apply(
  lambda x: x[:-4]
)
df_mask['mask_file'] = df_mask['mask_file'].apply(
  lambda x: os.path.join(data_dir, 'segmentations', x)
)
df = df_train.merge(df_mask, on='StudyInstanceUID', how='left')
df['image_folder'] = df['StudyInstanceUID'].apply(
  lambda x: os.path.join(data_dir, 'train_images', x)
)
display(df_train)
display(df_mask)
display(df)

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0


,mask_file,StudyInstanceUID
0,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.2243
1,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.3376
2,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.18906
3,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.18480
4,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.30640
...,...,...
82,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.24606
83,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.8574
84,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.27292
85,../data/raw/segmentations/1.2.826.0.1.3680043....,1.2.826.0.1.3680043.6376


,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,mask_file,image_folder
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.6200
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.2...
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.2...
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.1...
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,../data/raw/segmentations/1.2.826.0.1.3680043....,../data/raw/train_images/1.2.826.0.1.3680043.1363
...,...,...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1,NaN,../data/raw/train_images/1.2.826.0.1.3680043.2...
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1,NaN,../data/raw/train_images/1.2.826.0.1.3680043.4786
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.1...
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0,NaN,../data/raw/train_images/1.2.826.0.1.3680043.1...
